# Grating Coupler Simulation with FiberSource

[MEEP](https://meep.readthedocs.io/) is an open-source FDTD electromagnetic simulator. This notebook demonstrates using the `gsim.meep` **FiberSource** API to simulate fiber-to-chip coupling through a grating coupler.

Unlike S-parameter simulations that use eigenmode sources at waveguide ports, grating coupler simulations launch a **Gaussian beam from above** (simulating a fiber) and measure the power coupled into the waveguide port via eigenmode decomposition.

**Requirements:**

- UBC PDK: `uv pip install ubcpdk`
- [GDSFactory+](https://gdsfactory.com) account for cloud simulation

### Load a grating coupler from UBC PDK

In [ ]:
from ubcpdk import PDK, cells

PDK.activate()

c = cells.ebeam_gc_te1550()
c

### Configure simulation with FiberSource

Instead of `ModeSource` (which excites a waveguide eigenmode at a port), we use `FiberSource` to launch a Gaussian beam from above the grating. Key parameters:

- **beam_waist**: Gaussian beam waist radius (SMF-28 mode field diameter / 2 = 5.2 um)
- **angle_theta**: Fiber tilt angle from vertical (typically 8-15 degrees for grating couplers)
- **z_offset**: Distance above the chip surface to place the source plane
- **polarization**: TE or TM

The result is a `CouplingResult` instead of `SParameterResult`, giving coupling efficiency (CE) per port.

In [ ]:
from gsim import meep

sim = meep.Simulation()

sim.geometry(component=c, z_crop="auto")
sim.materials = {"si": 3.47, "SiO2": 1.44}

sim.source = meep.FiberSource(
    wavelength=1.55,
    wavelength_span=0.1,
    num_freqs=21,
    beam_waist=5.2,
    angle_theta=10.0,
    z_offset=2.0,
)

sim.monitors = ["o1"]
sim.domain(pml=1.0, margin=1.0, margin_z_above=3.0)
sim.solver(resolution=15, simplify_tol=0.01, save_animation=True, verbose_interval=5.0)
sim.solver.stop_after_sources(time=80)

print(sim.validate_config())

### Preview geometry

In [ ]:
sim.plot_2d(slices="xyz")

### Run simulation on cloud

In [ ]:
result = sim.run()

### Plot coupling efficiency

In [ ]:
result.plot(db=True)

In [ ]:
print("Peak coupling efficiency (dB):")
for port, ce_db in result.peak_ce.items():
    print(f"  {port}: {ce_db:.2f} dB")

In [ ]:
result.show_animation()